# 목적: visualize 및 평가 metric 계산

In [1]:
import argparse
import json
import os
import sys
import time

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning, NumbaWarning
import warnings
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaWarning)

import numpy as np
import torch
from pathlib import Path
import yaml
from det3d import torchie
from det3d.datasets import build_dataloader, build_dataset
from det3d.models import build_detector
from det3d.torchie import Config
from det3d.torchie.apis import (
    batch_processor,
    build_optimizer,
    get_root_logger,
    init_dist,
    set_random_seed,
    train_detector,
)
from det3d.torchie.trainer import load_checkpoint
import torch.distributed as dist
import subprocess

# 주피터 노트북 경로설정
os.chdir('../')

def parse_args():
    parser = argparse.ArgumentParser(description="Train a detector")
    parser.add_argument("--config", default="configs/etriInfra/pp/etriInfra_centerpoint_pp_02voxel_two_pfn_10sweep.py", help="train config file path")
    parser.add_argument("--work_dir", help="the dir to save logs and models")
    parser.add_argument("--resume_from", help="the checkpoint file to resume from")
    parser.add_argument(
        "--validate",
        action="store_true",
        help="whether to evaluate the checkpoint during training",
    )
    parser.add_argument(
        "--gpus",
        type=int,
        default=1,
        help="number of gpus to use " "(only applicable to non-distributed training)",
    )
    parser.add_argument("--seed", type=int, default=None, help="random seed")
    parser.add_argument(
        "--launcher",
        choices=["pytorch", "slurm"],
        default="pytorch",
        help="job launcher",
    )
    parser.add_argument("--local_rank", type=int, default=0)
    parser.add_argument(
        "--autoscale-lr",
        action="store_true",
        help="automatically scale lr with the number of gpus",
    )
    args = parser.parse_args(args=[])
    if "LOCAL_RANK" not in os.environ:
        os.environ["LOCAL_RANK"] = str(args.local_rank)

    return args
args = parse_args()
cfg = Config.fromfile(args.config)
# distribution 설정 안함
cfg.local_rank = args.local_rank 

no apex
No Tensorflow
Deformable Convolution not built!
No APEX!


In [2]:
# init logger before other steps
distributed = False
cfg.gpus = args.gpus
logger = get_root_logger(cfg.log_level)
logger.info("Distributed training: {}".format(distributed))
logger.info(f"torch.backends.cudnn.benchmark: {torch.backends.cudnn.benchmark}")

2024-04-12 15:07:51,924 - INFO - Distributed training: False
2024-04-12 15:07:51,925 - INFO - torch.backends.cudnn.benchmark: False


In [3]:
model = build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)

2024-04-12 15:07:52,260 - INFO - Finish RPN Initialization
2024-04-12 15:07:52,260 - INFO - num_classes: [2, 2, 1, 1, 2, 1]
2024-04-12 15:07:52,278 - INFO - Finish CenterHead Initialization


Use HM Bias:  -2.19


In [4]:
args.testset = False
dataset = build_dataset(cfg.data.val)

In [5]:
args.speed_test = True
data_loader = build_dataloader(
        dataset,
        batch_size=cfg.data.samples_per_gpu if not args.speed_test else 1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=distributed,
        shuffle=False,
    )

In [6]:
args.checkpoint = "modelzoo/etri3D_pointpillar/etri3D_latest.pth"
checkpoint = load_checkpoint(model, args.checkpoint, map_location="cpu")

In [7]:
model = model.cuda()
model.eval()
mode = "val"

In [8]:
logger.info(f"work dir: {args.work_dir}")
if cfg.local_rank == 0:
    prog_bar = torchie.ProgressBar(len(data_loader.dataset) // cfg.gpus)

detections = {}
cpu_device = torch.device("cpu")

start = time.time()

start = int(len(dataset) / 3)
end = int(len(dataset) * 2 /3)

time_start = 0 
time_end = 0 

2024-04-12 15:07:54,719 - INFO - work dir: None


[                              ] 0/3000, elapsed: 0s, ETA:

In [9]:
for i, data_batch in enumerate(data_loader):
    if i == start:
        torch.cuda.synchronize()
        time_start = time.time()

    if i == end:
        torch.cuda.synchronize()
        time_end = time.time()

    with torch.no_grad():
        outputs = batch_processor(
            model, data_batch, train_mode=False, local_rank=args.local_rank,
        )
    break

/home/jaelee/anaconda3/envs/centerpoint/lib/python3.8/site-packages/numpy/core/shape_base.py:420: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arrays = [asanyarray(arr) for arr in arrays]
/home/jaelee/anaconda3/envs/centerpoint/lib/python3.8/site-packages/numpy/core/shape_base.py:420: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arrays = [asanyarray(arr) for arr in arrays]
/home/jaelee/anaconda3/envs/centerpoint/lib/python3.8/site-packages/numpy/core/shape_base.py:420: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (whi

In [10]:
data_batch

{'metadata': [{'image_prefix': PosixPath('data/etri3Dobj_infra_edge'),
   'num_point_features': 3}],
 'points': [tensor([[-2.9951e-04,  1.4299e-01,  3.8175e-02],
          [ 1.2478e-04,  1.4299e-01,  3.8175e-02],
          [ 6.2054e-04,  1.5458e-01,  4.1271e-02],
          ...,
          [-1.7872e-03,  1.3837e-01, -3.9827e-02],
          [-1.3901e-03,  1.4222e-01, -4.0933e-02],
          [-8.9228e-04,  1.3454e-01, -3.8721e-02]])],
 'voxels': tensor([[[-2.9951e-04,  1.4299e-01,  3.8175e-02],
          [-6.0129e-02,  1.9175e-01,  5.3652e-02],
          [-6.1816e-02,  1.9933e-01,  5.5716e-02],
          ...,
          [-3.8855e-02,  1.6951e-01,  4.6430e-02],
          [-3.7470e-02,  1.6586e-01,  4.5398e-02],
          [-3.1911e-02,  1.4335e-01,  3.9207e-02]],
 
         [[ 1.2478e-04,  1.4299e-01,  3.8175e-02],
          [ 6.2054e-04,  1.5458e-01,  4.1271e-02],
          [ 1.1891e-03,  1.6618e-01,  4.4366e-02],
          ...,
          [ 9.0158e-03,  1.6593e-01,  4.4366e-02],
          [ 

In [11]:
len(data_batch['anno_box'])

6

In [14]:
data_batch['anno_cls']

array([[array([1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32),
        array([], dtype=int32), array([], dtype=int32),
        array([], dtype=int32), array([], dtype=int32),
        array([], dtype=int32)]], dtype=object)

In [15]:
outputs[0]['box3d_lidar'].cpu().numpy().shape

(15, 7)

In [16]:
len(outputs)

1

In [17]:
outputs[0]

{'box3d_lidar': tensor([[-2.0572e+01, -4.2772e+00, -5.6578e-01,  2.2512e+00,  4.6266e+00,
           1.6879e+00, -4.4454e-01],
         [-3.3328e+01, -2.6246e+00,  4.1864e-03,  2.1117e+00,  4.5908e+00,
           1.6341e+00, -2.8629e-01],
         [-3.0738e+01, -7.5428e-02,  1.4909e-01,  2.1568e+00,  4.0811e+00,
           1.5518e+00, -2.6346e-01],
         [-2.4313e+01, -6.4297e+00, -4.5093e-01,  2.0369e+00,  4.7302e+00,
           1.4803e+00, -4.8538e-01],
         [ 8.5508e+00, -1.4463e+01, -1.0313e+00,  2.1285e+00,  4.5201e+00,
           1.6315e+00, -2.3890e+00],
         [-2.6394e+01,  4.1298e+00,  4.9087e-01,  2.1077e+00,  4.3319e+00,
           1.9290e+00, -3.8688e-01],
         [ 4.3494e+01,  3.2174e+01, -1.2120e+00,  2.1740e+00,  5.3026e+00,
           1.7410e+00, -2.3136e+00],
         [-1.1879e+01, -4.1863e+01, -9.8651e-01,  2.1246e+00,  4.1871e+00,
           1.6877e+00,  2.7386e+00],
         [-2.3597e+01, -1.3271e+01, -8.2382e-01,  6.0167e-01,  1.4842e+00,
           1.4

In [18]:
output = outputs[0]
for k, v in output.items():
    print(f"{k}")

box3d_lidar
scores
label_preds
metadata


In [19]:
detections = []
for k, v in output.items():
    if k not in [
        "metadata",
    ]:
        output[k] = v.to(cpu_device)
    detections.append(output)

In [20]:
output['box3d_lidar'].cpu().numpy()

array([[-2.0572489e+01, -4.2772408e+00, -5.6578290e-01,  2.2511895e+00,
         4.6265812e+00,  1.6878871e+00, -4.4454461e-01],
       [-3.3327682e+01, -2.6246338e+00,  4.1864216e-03,  2.1116798e+00,
         4.5908461e+00,  1.6340932e+00, -2.8629452e-01],
       [-3.0738068e+01, -7.5428009e-02,  1.4908600e-01,  2.1568010e+00,
         4.0810599e+00,  1.5518138e+00, -2.6345658e-01],
       [-2.4312590e+01, -6.4296608e+00, -4.5093194e-01,  2.0368633e+00,
         4.7302055e+00,  1.4802680e+00, -4.8537868e-01],
       [ 8.5507698e+00, -1.4463158e+01, -1.0312934e+00,  2.1284766e+00,
         4.5201240e+00,  1.6314881e+00, -2.3890283e+00],
       [-2.6394234e+01,  4.1297989e+00,  4.9087492e-01,  2.1077068e+00,
         4.3318663e+00,  1.9290257e+00, -3.8688421e-01],
       [ 4.3493717e+01,  3.2173809e+01, -1.2119828e+00,  2.1739845e+00,
         5.3026314e+00,  1.7410246e+00, -2.3136003e+00],
       [-1.1878700e+01, -4.1862572e+01, -9.8651147e-01,  2.1245551e+00,
         4.1870604e+00,  

In [38]:
output['label_preds'].cpu().numpy()

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 6, 6, 6, 6, 7])

In [40]:
output['scores'].cpu().numpy()

array([0.97432446, 0.9651002 , 0.9556864 , 0.80238724, 0.7886445 ,
       0.7079947 , 0.6652666 , 0.51221067, 0.32024455, 0.13874333,
       0.2061549 , 0.11421622, 0.11346538, 0.10587288, 0.10296192],
      dtype=float32)

In [21]:
data_batch['points'][0].cpu().numpy()

array([[-2.9950603e-04,  1.4299141e-01,  3.8175460e-02],
       [ 1.2478379e-04,  1.4299168e-01,  3.8175460e-02],
       [ 6.2054489e-04,  1.5458441e-01,  4.1270766e-02],
       ...,
       [-1.7872249e-03,  1.3837129e-01, -3.9827053e-02],
       [-1.3901073e-03,  1.4222001e-01, -4.0933359e-02],
       [-8.9227577e-04,  1.3453589e-01, -3.8720746e-02]], dtype=float32)

### visualize


In [22]:
def pt_to_xyz(fp):
    x = []
    y = []
    z = []
    # fp = np.fromfile(fname, dtype=np.float32)
    # fp = fp.reshape(-1,3)
    for i in range(fp.shape[0]):
        x.append(fp[i][0])
        y.append(fp[i][1])
        z.append(fp[i][2])
    return x,y,z

def writePCDFile(fname,x,y,z):
    numPoints= len(x)
    with open(fname, 'w') as fp:
        fp.write("VERSION 0.7\n")
        fp.write("FIELDS x y z\n")
        fp.write("SIZE 4 4 4\n")
        fp.write("TYPE F F F\n")
        fp.write("WIDTH "+str(numPoints)+"\n")
        fp.write("HEIGHT 1\n")
        fp.write("POINTS "+str(numPoints)+"\n")
        fp.write("DATA ascii\n")
        for index in range(numPoints):
            txtLine = "{} {} {}\n".format(x[index],y[index],z[index] )
            fp.write(txtLine)
        pass


In [20]:
rt = "/home/jaelee/objdect/CenterPoint/etri3Deval"
x,y,z =pt_to_xyz(data_batch['points'][0].cpu().numpy())
idx_p = 0
pth = Path(f"ex{idx_p}")
pcd_name = rt / pth.with_suffix('.pcd')
writePCDFile(pcd_name,x,y,z)

In [23]:
pred_boxes = output['box3d_lidar'].cpu().numpy()

In [41]:
import open3d as o3d
from open3d.web_visualizer import draw
points_v = data_batch["points"][0]
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=2, origin=[0,0,0])
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_v)
entities_to_draw = [pcd, mesh_frame]
# for idx in range(gt_boxes.shape[0]):
# idx = 0
for idx in range(len(pred_boxes)):
        if output['scores'].cpu().numpy()[idx] <= 0.2:
                continue
        translation = pred_boxes[idx][:3]
        w, l, h = pred_boxes[idx][3], pred_boxes[idx][4], pred_boxes[idx][5]
        rotation = pred_boxes[idx][-1]

        bounding_box = np.array([
                        [-l/2, -l/2, l/2, l/2, -l/2, -l/2, l/2, l/2],
                        [w/2, -w/2, -w/2, w/2, w/2, -w/2, -w/2, w/2],
                        [-h/2, -h/2, -h/2, -h/2, h/2, h/2, h/2, h/2]]) 
        rotation_matrix = np.array([
                [np.cos(rotation), -np.sin(rotation), 0.0],
                [np.sin(rotation), np.cos(rotation), 0.0],
                [0.0, 0.0, 1.0]])
        eight_points = np.tile(translation, (8, 1))

        corner_box = np.dot(rotation_matrix, bounding_box) + eight_points.transpose()
        boxes3d_pts = corner_box.transpose()
        boxes3d_pts = boxes3d_pts.T
        boxes3d_pts = o3d.utility.Vector3dVector(boxes3d_pts.T)
        box = o3d.geometry.OrientedBoundingBox.create_from_points(boxes3d_pts)
        box.color = [1, 0, 0]           #Box color would be red box.color = [R,G,B]
        entities_to_draw.append(box)
draw([*entities_to_draw])


[Open3D INFO] Window window_3 created.


WebVisualizer(window_uid='window_3')

[Open3D INFO] Sending init frames to window_3.


[4687:067][751005] (stun_port.cc:96): Binding request timed out from 10.42.0.x:51437 (enp5s0)
[4687:132][751005] (stun_port.cc:96): Binding request timed out from 10.42.0.x:51437 (enp5s0)


In [37]:
gt_boxes = []
for ts in data_batch['anno_box']:
    ts = ts[0]
    gt_boxes.extend(ts.cpu().numpy())
points_v = data_batch["points"][0]
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=2, origin=[0,0,0])
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_v)
entities_to_draw = [pcd, mesh_frame]
# for idx in range(gt_boxes.shape[0]):
# idx = 0
for idx in range(len(gt_boxes)):
        translation = gt_boxes[idx][:3]
        w, l, h = gt_boxes[idx][3], gt_boxes[idx][4], gt_boxes[idx][5]
        rotation = gt_boxes[idx][-1]

        bounding_box = np.array([
                        [-l/2, -l/2, l/2, l/2, -l/2, -l/2, l/2, l/2],
                        [w/2, -w/2, -w/2, w/2, w/2, -w/2, -w/2, w/2],
                        [-h/2, -h/2, -h/2, -h/2, h/2, h/2, h/2, h/2]]) 
        rotation_matrix = np.array([
                [np.cos(rotation), -np.sin(rotation), 0.0],
                [np.sin(rotation), np.cos(rotation), 0.0],
                [0.0, 0.0, 1.0]])
        eight_points = np.tile(translation, (8, 1))

        corner_box = np.dot(rotation_matrix, bounding_box) + eight_points.transpose()
        boxes3d_pts = corner_box.transpose()
        boxes3d_pts = boxes3d_pts.T
        boxes3d_pts = o3d.utility.Vector3dVector(boxes3d_pts.T)
        box = o3d.geometry.OrientedBoundingBox.create_from_points(boxes3d_pts)
        box.color = [0, 1, 0]           #Box color would be red box.color = [R,G,B]
        entities_to_draw.append(box)
draw([*entities_to_draw])


[Open3D INFO] Window window_2 created.


WebVisualizer(window_uid='window_2')

[Open3D INFO] Sending init frames to window_2.


[334:119][751005] (stun_port.cc:96): Binding request timed out from 10.42.0.x:38251 (enp5s0)
[334:219][751005] (stun_port.cc:96): Binding request timed out from 10.42.0.x:38251 (enp5s0)
[352:797][751005] (stun_port.cc:96): Binding request timed out from 10.42.0.x:34992 (enp5s0)
[352:798][751005] (stun_port.cc:96): Binding request timed out from 10.42.0.x:34992 (enp5s0)


### metric evaluation

In [45]:
cfg.tasks

[{'num_class': 2, 'class_names': ['car', 'personal_mobility']},
 {'num_class': 2, 'class_names': ['truck', 'construction_vehicle']},
 {'num_class': 1, 'class_names': ['bus']},
 {'num_class': 1, 'class_names': ['ground_animal']},
 {'num_class': 2, 'class_names': ['motorcycle', 'bicycle']},
 {'num_class': 1, 'class_names': ['pedestrian']}]

In [43]:
data_batch['anno_cls']

array([[array([1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32),
        array([], dtype=int32), array([], dtype=int32),
        array([], dtype=int32), array([], dtype=int32),
        array([], dtype=int32)]], dtype=object)

In [46]:
# cls_int2name: cls 고유번호에서 이름으로 변환하는 딕셔너리
cls_int2name = {}
idx = 0
for i in range(len(cfg.tasks)):
    head_classes = cfg.tasks[i]
    class_names = head_classes['class_names']
    for j, name in enumerate(class_names):
        cls_int2name[idx+j] = name
    idx += len(class_names)
cls_int2name

{0: 'car',
 1: 'personal_mobility',
 2: 'truck',
 3: 'construction_vehicle',
 4: 'bus',
 5: 'ground_animal',
 6: 'motorcycle',
 7: 'bicycle',
 8: 'pedestrian'}

In [49]:
data_batch['anno_cls'][0]

array([array([1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32),
       array([], dtype=int32), array([], dtype=int32),
       array([], dtype=int32), array([], dtype=int32),
       array([], dtype=int32)], dtype=object)

In [52]:
# data_batch['anno_cls'] 변환
input_cls_id = []
idx = 0
for i in range(data_batch['anno_cls'][0].shape[0]):
    head_classes = cfg.tasks[i]
    class_names = head_classes['class_names']
    head_class_ids = data_batch['anno_cls'][0][i]
    for j in range(len(head_class_ids)):
        _id = head_class_ids[j] - 1
        input_cls_id.append(idx+_id)
    idx += len(class_names)
input_cls_id = np.array(input_cls_id)
input_cls_id

array([0, 0, 0, 0, 0, 0, 0, 0, 0])

In [57]:
import glob
import os
import sys
import time
import numpy as np
import matplotlib.pyplot as plt

In [71]:
distance_threshold = 1.0
save_loc = 'work_dirs/'
distance_threshold_sq = distance_threshold**2
score_threshold = min_score = 0.0
max_range = max_range = 0.0
classes = [v for v in cls_int2name.values()]
total_N_pos = 0
results_dict = {}
for single_class in classes:
    class_dict = {}
    class_dict['class'] = single_class
    class_dict['T_p'] = np.empty((0, 8))
    class_dict['gt'] = np.empty((0, 7))
    class_dict['total_N_pos'] = 0
    class_dict['result'] = np.empty((0, 2))
    class_dict['precision'] = []
    class_dict['recall'] = []
    results_dict[single_class] = class_dict
time = time.time()
# self.evaluate(pred_label_path, gt_label_path, label_format)
# file_parsing: class_id, x, y, z, l, w, h, r 순으로 배열 구성

def output_to_evalform(cls_array, box_array, pred_score = None):
    # class_id, x, y, z, l, w, h, r 순으로 배열 구성
    N = cls_array.shape[0]
    if pred_score is not None:
        o_ary = np.zeros((N, 9))
    else:
        o_ary = np.zeros((N, 8))
    for i in range(N):
        o_ary[i][0] = cls_array[i]
        o_ary[i][1] = box_array[i][0]
        o_ary[i][2] = box_array[i][1]
        o_ary[i][3] = box_array[i][2]
        o_ary[i][4] = box_array[i][4]
        o_ary[i][5] = box_array[i][3]
        o_ary[i][6] = box_array[i][5]
        o_ary[i][7] = box_array[i][-1]
        if pred_score is not None:
            o_ary[i][8] = pred_score[i]
    return o_ary
predictions = output_to_evalform(output['label_preds'].cpu().numpy(), output['box3d_lidar'].cpu().numpy(), output['scores'].cpu().numpy())
ground_truth = output_to_evalform(input_cls_id, np.stack(gt_boxes))

AttributeError: 'float' object has no attribute 'time'

In [78]:
def match_pairs(pred_label, gt_label):
    true_preds = np.empty((0, 8))
    corresponding_gt = np.empty((0, 7))
    result_score = np.empty((0, 2))
    # Initialize matching loop
    match_incomplete = True
    while match_incomplete and gt_label.shape[0] > 0:
        match_incomplete = False
        for gt_idx, single_gt_label in enumerate(gt_label):
            # Check is any prediction is in range
            distance_sq_array = (single_gt_label[0] - pred_label[:, 0])**2 + (single_gt_label[1] - pred_label[:, 1])**2
            # If there is a prediction in range, pick closest
            if np.any(distance_sq_array < distance_threshold_sq):
                min_idx = np.argmin(distance_sq_array)
                # Store true prediction
                true_preds = np.vstack((true_preds, pred_label[min_idx, :].reshape(-1, 1).T))
                corresponding_gt = np.vstack((corresponding_gt, gt_label[gt_idx]))

                # Store score for mAP
                result_score = np.vstack((result_score, np.array([[1, pred_label[min_idx, 7]]])))

                # Remove prediction and gt then reset loop
                pred_label = np.delete(pred_label, obj=min_idx, axis=0)
                gt_label = np.delete(gt_label, obj=gt_idx, axis=0)
                match_incomplete = True
                break
    # If there were any false detections, add them.
    if pred_label.shape[0] > 0:
        false_positives = np.zeros((pred_label.shape[0], 2))
        false_positives[:, 1] = pred_label[:, 7]
        result_score = np.vstack((result_score, false_positives))
    return true_preds, corresponding_gt, result_score

def eval_pair(pred_label, gt_label, results_dict):
    ## Check
    assert pred_label.shape[1] == 9
    assert gt_label.shape[1] == 8

    ## Threshold score
    if pred_label.shape[0] > 0:
        pred_label = pred_label[pred_label[:, 8].astype(np.float) > score_threshold, :]

    for single_class in classes:
        # get all pred labels, order by score
        class_pred_label = pred_label[np.char.lower(pred_label[:, 0].astype(str)) == single_class, 1:]
        score = class_pred_label[:, 7].astype(np.float)
        class_pred_label = class_pred_label[(-score).argsort(), :].astype(np.float) # sort decreasing

        # add gt label length to total_N_pos
        class_gt_label = gt_label[np.char.lower(gt_label[:, 0].astype(str)) == single_class, 1:].astype(np.float)
        results_dict[single_class]['total_N_pos'] += class_gt_label.shape[0]

        # match pairs
        pred_array, gt_array, result_score_pair = match_pairs(class_pred_label, class_gt_label)
        
        # add to existing results
        results_dict[single_class]['T_p'] = np.vstack((results_dict[single_class]['T_p'], pred_array))
        results_dict[single_class]['gt'] = np.vstack((results_dict[single_class]['gt'], gt_array))
        results_dict[single_class]['result'] = np.vstack((results_dict[single_class]['result'], result_score_pair))
    return results_dict
results_dict = eval_pair(predictions, ground_truth, results_dict)

/tmp/ipykernel_750375/3514758787.py:41: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pred_label = pred_label[pred_label[:, 8].astype(np.float) > score_threshold, :]


UnboundLocalError: local variable 'class_gt_label' referenced before assignment

In [73]:
results_dict

{'car': {'class': 'car',
  'T_p': array([], shape=(0, 8), dtype=float64),
  'gt': array([], shape=(0, 7), dtype=float64),
  'total_N_pos': 0,
  'result': array([], shape=(0, 2), dtype=float64),
  'precision': [],
  'recall': []},
 'personal_mobility': {'class': 'personal_mobility',
  'T_p': array([], shape=(0, 8), dtype=float64),
  'gt': array([], shape=(0, 7), dtype=float64),
  'total_N_pos': 0,
  'result': array([], shape=(0, 2), dtype=float64),
  'precision': [],
  'recall': []},
 'truck': {'class': 'truck',
  'T_p': array([], shape=(0, 8), dtype=float64),
  'gt': array([], shape=(0, 7), dtype=float64),
  'total_N_pos': 0,
  'result': array([], shape=(0, 2), dtype=float64),
  'precision': [],
  'recall': []},
 'construction_vehicle': {'class': 'construction_vehicle',
  'T_p': array([], shape=(0, 8), dtype=float64),
  'gt': array([], shape=(0, 7), dtype=float64),
  'total_N_pos': 0,
  'result': array([], shape=(0, 2), dtype=float64),
  'precision': [],
  'recall': []},
 'bus': {'clas

In [ ]:
# 출처: https://github.com/jacoblambert/3d_lidar_detection_evaluation/blob/master/nuscenes_eval_core.py
from label_parser import LabelParser

class NuScenesEval:
    def __init__(self, pred_label_path, gt_label_path, label_format, save_loc,
                 classes=['car', 'pedestrian', 'cyclist'],
                 distance_threshold=1.0,
                 min_score=0.0,
                 max_range=0.0):

        # Initialize
        self.save_loc = save_loc
        self.distance_threshold_sq = distance_threshold**2
        self.score_threshold = min_score
        self.max_range = max_range
        self.classes = classes
        self.total_N_pos = 0
        self.results_dict = {}
        for single_class in classes:
            class_dict = {}
            class_dict['class'] = single_class
            class_dict['T_p'] = np.empty((0, 8))
            class_dict['gt'] = np.empty((0, 7))
            class_dict['total_N_pos'] = 0
            class_dict['result'] = np.empty((0, 2))
            class_dict['precision'] = []
            class_dict['recall'] = []
            self.results_dict[single_class] = class_dict
        # Format
        if pred_label_path[-1] is not "/":
            pred_label_path += "/"
        if gt_label_path[-1] is not "/":
            gt_label_path += "/"
        # Run
        self.time = time.time()
        self.evaluate(pred_label_path, gt_label_path, label_format)

    def evaluate(self, pred_path, gt_path, label_format):
        pred_file_list = glob.glob(pred_path + "*")
        pred_file_list.sort()
        gt_file_list = glob.glob(gt_path + "*")
        gt_file_list.sort()
        num_examples = len(pred_file_list)
        print("Starting evaluation for {} file predictions".format(num_examples))
        print("--------------------------------------------")

        ## Check missing files
        print("Confirmation prediction ground truth file pairs.")
        for pred_fn in pred_file_list:
            if (gt_path + os.path.basename(pred_fn)) not in gt_file_list:
                print("Error loading labels: gt label for pred label {} was not found.".format(
                    os.path.basename(pred_fn)))
                sys.exit(1)

        ## Evaluate matches
        print("Evaluation examples")
        file_parsing = LabelParser(label_format)
        for i, pred_fn in enumerate(pred_file_list):
            # print("\r", i+1, "/", num_examples, end="")
            gt_fn = gt_path + os.path.basename(pred_fn)
            predictions = file_parsing.parse_label(pred_fn, prediction=True)
            ground_truth = file_parsing.parse_label(gt_fn, prediction=False)
        # Filter range
            if self.max_range > 0:
                predictions, ground_truth = self.filter_by_range(predictions, ground_truth, range=self.max_range)
            self.eval_pair(predictions, ground_truth)
        print("\nDone!")
        print("----------------------------------")
        ## Calculate
        for single_class in self.classes:
            class_dict = self.results_dict[single_class]
            print("Calculating metrics for {} class".format(single_class))
            print("----------------------------------")
            print("Number of ground truth labels: ", class_dict['total_N_pos'])
            print("Number of detections:  ", class_dict['result'].shape[0])
            print("Number of true positives:  ", np.sum(class_dict['result'][:, 0] == 1))
            print("Number of false positives:  ", np.sum(class_dict['result'][:, 0] == 0))
            if class_dict['total_N_pos'] == 0:
                print("No detections for this class!")
                print(" ")
                continue
            ## AP
            self.compute_ap_curve(class_dict)
            mean_ap = self.compute_mean_ap(class_dict['precision'], class_dict['recall'])
            print('Mean AP: %.3f ' % mean_ap)
            f1 = self.compute_f1_score(class_dict['precision'], class_dict['recall'])
            print('F1 Score: %.3f ' % f1)
            print(' ')
            ## Positive Thresholds
            # ATE 2D
            ate2d = self.compute_ate2d(class_dict['T_p'], class_dict['gt'])
            print('Average 2D Translation Error [m]:  %.4f ' % ate2d)
            # ATE 3D
            ate3d = self.compute_ate3d(class_dict['T_p'], class_dict['gt'])
            print('Average 3D Translation Error [m]:  %.4f ' % ate3d)
            # ASE
            ase = self.compute_ase(class_dict['T_p'], class_dict['gt'])
            print('Average Scale Error:  %.4f ' % ase)
            # AOE
            aoe = self.compute_aoe(class_dict['T_p'], class_dict['gt'])
            print('Average Orientation Error [rad]:  %.4f ' % aoe)
            print(" ")
        self.time = float(time.time() - self.time)
        print("Total evaluation time: %.5f " % self.time)

    def compute_ap_curve(self, class_dict):
        t_pos = 0
        class_dict['precision'] = np.ones(class_dict['result'].shape[0]+2)
        class_dict['recall'] = np.zeros(class_dict['result'].shape[0]+2)
        sorted_detections = class_dict['result'][(-class_dict['result'][:, 1]).argsort(), :]
        print(sorted_detections.shape)
        for i, (result_bool, result_score) in enumerate(sorted_detections):
            if result_bool == 1:
                t_pos += 1
            class_dict['precision'][i+1] = t_pos / (i + 1)
            class_dict['recall'][i+1] = t_pos / class_dict['total_N_pos']
        class_dict['precision'][i+2] = 0
        class_dict['recall'][i+2] = class_dict['recall'][i+1]

        ## Plot
        plt.figure()
        plt.plot(class_dict['recall'], class_dict['precision'])
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision Recall curve for {} Class'.format(class_dict['class']))
        plt.xlim([0, 1])
        plt.ylim([0, 1.05])
        plt.savefig(self.save_loc + class_dict['class'] + "_pr_curve.png")

    def compute_f1_score(self, precision, recall):
        p, r = precision[(precision+recall) > 0], recall[(precision+recall) > 0]
        f1_scores = 2 * p * r / (p + r)
        return np.max(f1_scores)

    def compute_mean_ap(self, precision, recall, precision_threshold=0.0, recall_threshold=0.0):
        mean_ap = 0
        threshold_mask = np.logical_and(precision > precision_threshold,
                                        recall > recall_threshold)
        # calculate mean AP
        precision = precision[threshold_mask]
        recall = recall[threshold_mask]
        recall_diff = np.diff(recall)
        precision_diff = np.diff(precision)
        # Square area under curve based on i+1 precision, then linear difference in precision
        mean_ap = np.sum(precision[1:]*recall_diff + recall_diff*precision_diff/2)
        # We need to divide by (1-recall_threshold) to make the max possible mAP = 1. In practice threshold by the first
        # considered recall value (threshold = 0.1 -> first considered value may be = 0.1123)
        mean_ap = mean_ap/(1-recall[0])
        return mean_ap

    def compute_ate2d(self, predictions, ground_truth):
        # euclidean distance 3d
        mean_ate2d = np.mean(np.sqrt((predictions[:, 0] - ground_truth[:, 0])**2 +
                                     (predictions[:, 1] - ground_truth[:, 1])**2))
        return mean_ate2d

    def compute_ate3d(self, predictions, ground_truth):
        # euclidean distance 2d
        mean_ate3d = np.mean(np.sqrt((predictions[:, 0] - ground_truth[:, 0]) ** 2 +
                                     (predictions[:, 1] - ground_truth[:, 1]) ** 2 +
                                     (predictions[:, 2] - ground_truth[:, 2]) ** 2))
        return mean_ate3d

    def compute_ase(self, predictions, ground_truth):
        # simplified iou where boxes are centered and aligned with eachother
        pred_vol = predictions[:, 3:6]
        gt_vol = ground_truth[:, 3:6]
        iou3d = np.mean(1 - np.prod(np.minimum(pred_vol, gt_vol), axis=1)/np.prod(np.maximum(pred_vol, gt_vol), axis=1))
        return iou3d

    def compute_aoe(self, predictions, ground_truth):
        err = ground_truth[:,6] - predictions[:,6]
        aoe = np.mean(np.abs((err + np.pi) % (2*np.pi) - np.pi))
        return aoe

    def eval_pair(self, pred_label, gt_label):
        ## Check
        assert pred_label.shape[1] == 9
        assert gt_label.shape[1] == 8

        ## Threshold score
        if pred_label.shape[0] > 0:
            pred_label = pred_label[pred_label[:, 8].astype(np.float) > self.score_threshold, :]

        for single_class in self.classes:
            # get all pred labels, order by score
            class_pred_label = pred_label[np.char.lower(pred_label[:, 0].astype(str)) == single_class, 1:]
            score = class_pred_label[:, 7].astype(np.float)
            class_pred_label = class_pred_label[(-score).argsort(), :].astype(np.float) # sort decreasing

            # add gt label length to total_N_pos
            class_gt_label = gt_label[np.char.lower(gt_label[:, 0].astype(str)) == single_class, 1:].astype(np.float)
            self.results_dict[single_class]['total_N_pos'] += class_gt_label.shape[0]

            # match pairs
            pred_array, gt_array, result_score_pair = self.match_pairs(class_pred_label, class_gt_label)

            # add to existing results
            self.results_dict[single_class]['T_p'] = np.vstack((self.results_dict[single_class]['T_p'], pred_array))
            self.results_dict[single_class]['gt'] = np.vstack((self.results_dict[single_class]['gt'], gt_array))
            self.results_dict[single_class]['result'] = np.vstack((self.results_dict[single_class]['result'],
                                                                   result_score_pair))

    def match_pairs(self, pred_label, gt_label):
        true_preds = np.empty((0, 8))
        corresponding_gt = np.empty((0, 7))
        result_score = np.empty((0, 2))
        # Initialize matching loop
        match_incomplete = True
        while match_incomplete and gt_label.shape[0] > 0:
            match_incomplete = False
            for gt_idx, single_gt_label in enumerate(gt_label):
                # Check is any prediction is in range
                distance_sq_array = (single_gt_label[0] - pred_label[:, 0])**2 + (single_gt_label[1] - pred_label[:, 1])**2
                # If there is a prediction in range, pick closest
                if np.any(distance_sq_array < self.distance_threshold_sq):
                    min_idx = np.argmin(distance_sq_array)
                    # Store true prediction
                    true_preds = np.vstack((true_preds, pred_label[min_idx, :].reshape(-1, 1).T))
                    corresponding_gt = np.vstack((corresponding_gt, gt_label[gt_idx]))

                    # Store score for mAP
                    result_score = np.vstack((result_score, np.array([[1, pred_label[min_idx, 7]]])))

                    # Remove prediction and gt then reset loop
                    pred_label = np.delete(pred_label, obj=min_idx, axis=0)
                    gt_label = np.delete(gt_label, obj=gt_idx, axis=0)
                    match_incomplete = True
                    break

        # If there were any false detections, add them.
        if pred_label.shape[0] > 0:
            false_positives = np.zeros((pred_label.shape[0], 2))
            false_positives[:, 1] = pred_label[:, 7]
            result_score = np.vstack((result_score, false_positives))
        return true_preds, corresponding_gt, result_score

    def filter_by_range(self, pred_label, gt_label, range=0):
        pred_dist = np.linalg.norm(pred_label[:, 1:4].astype(np.float32), axis=1) < range
        gt_dist = np.linalg.norm(gt_label[:, 1:4].astype(np.float32), axis=1) < range
        return pred_label[pred_dist, :], gt_label[gt_dist, :]